In [10]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
data_folder_path = '/content/drive/My Drive/adAuditing/personas_new/age'

In [12]:
import json, os

reference: https://www.analyticsvidhya.com/blog/2022/03/keyword-extraction-methods-from-documents-in-nlp/

In [14]:
extract_persona_info()

[{'pid': '4b81a719-4d63-4a0d-92be-5087c71a40b0_a120fce9-2511-422b-9c80-cb739e59ef6a',
  'first_name': 'Samantha',
  'last_name': 'Miller',
  'age': '45',
  'profile': "Samantha Miller is a 45-year-old Caucasian female living in 456 Oak St, Portland, Oregon 97201. She speaks English and holds a bachelor's degree in Psychology. Samantha's date of birth is 09/14/1979. She works full-time in the healthcare industry and is employed by a medium-sized employer (51-250 employees). She is a mental health counselor, with an annual income of $60,000. Samantha is a homeowner. She is divorced and has one teenage child. In her free time, she enjoys reading self-help books and watching classic movies. When using her computer, she prefers using a desktop for its larger screen and ease of navigation. On the internet, she likes to shop for household items and read articles on mental wellness and personal development."},
 {'pid': '4b81a719-4d63-4a0d-92be-5087c71a40b0_199bea01-2cba-4381-9980-fad5a40da13e'

In [15]:
import spacy
from collections import Counter
from string import punctuation
nlp = spacy.load("en_core_web_sm")
# nlp = en_core_web_sm.load()
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN']
    doc = nlp(text.lower())
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            result.append(token.text)
    return result


In [16]:
def extractKeywords():
    res = {}
    for file_name in os.listdir(data_folder_path):
        if not file_name.endswith('.json'): continue
        with open(os.path.join(data_folder_path, file_name), 'r') as file:
            data = json.load(file)['data']
            pid = file_name.split('.')[0]
            res[pid] = {}
            ignore = {data["first_name"].lower(), data["last_name"].lower()}
            for bh in data['browsing_history']:
                keywords = get_hotwords(bh['title'])
                for kw in keywords:
                    if kw.lower() in ignore: continue
                    if kw in res[pid]:
                        res[pid][kw]['browsing_history'].add(bh['id'])
                    else:
                        res[pid][kw] = {'browsing_history': {bh['id']}, 'schedule': set(), 'facebookPostsList': set()}
            for sch in data['schedule']:
                keywords = get_hotwords(sch['event'])
                for kw in keywords:
                    if kw.lower() in ignore: continue
                    if kw in res[pid]:
                        res[pid][kw]['schedule'].add(sch['id'])
                    else:
                        res[pid][kw] = {'browsing_history': set(), 'schedule': {sch['id']}, 'facebookPostsList': set()}
    print(res)
    return res

# {pid1 : {kw1 : {'browsing_history' : {id1, id2...}, {'schedule' : {id1, id2...}}, {'facebookPostsList' : {id1, id2...}}}, kw2: ...}, pid2 : {...}, pid3 : {...}}


            # for pc in data['facebookPostsList']:
            #   passage_embedding = model.encode(pc['content'])
            #   if util.dot_score(query_embedding, passage_embedding) > THRESHOLD:
            #     result['facebookPostsList'].append(pc['id'])


In [17]:
all_persona_keywords = extractKeywords()

{'4b81a719-4d63-4a0d-92be-5087c71a40b0_a120fce9-2511-422b-9c80-cb739e59ef6a': {'psychology': {'browsing_history': {0, 35, 38, 10, 11, 12, 45, 46, 30}, 'schedule': set(), 'facebookPostsList': set()}, 'today': {'browsing_history': {0, 35, 45, 46}, 'schedule': set(), 'facebookPostsList': set()}, 'therapist': {'browsing_history': {0}, 'schedule': set(), 'facebookPostsList': set()}, 'mental': {'browsing_history': {1, 4, 5, 13, 45, 22, 55, 59, 29}, 'schedule': set(), 'facebookPostsList': set()}, 'health': {'browsing_history': {1, 4, 5, 13, 22, 55, 29}, 'schedule': set(), 'facebookPostsList': set()}, 'blogs': {'browsing_history': {1, 13}, 'schedule': set(), 'facebookPostsList': set()}, 'stress': {'browsing_history': {8, 2, 27}, 'schedule': set(), 'facebookPostsList': set()}, 'anxiety': {'browsing_history': {2}, 'schedule': set(), 'facebookPostsList': set()}, 'best': {'browsing_history': {3, 6, 70, 40, 9, 60, 17, 50, 21, 23, 25, 28, 31}, 'schedule': set(), 'facebookPostsList': set()}, 'online'

In [18]:
def get_common_keywords(all_keywords):
    common_keywords = None

    for persona in all_keywords:
        keywords = set(all_keywords[persona].keys())
        if common_keywords is None:
            common_keywords = keywords
        else:
            common_keywords = common_keywords.intersection(keywords)
    print(f"common kws: {len(common_keywords)}")
    return common_keywords

In [19]:
def get_distinct_keywords(all_keywords, common_keywords, person_id_lst):
    res = {}
    for pid in person_id_lst:
        res[pid] = set(all_keywords[pid].keys()) - common_keywords
    return res
    # {pid1: {w1, w2, ...}, pid2: {...}, ...}


In [20]:
pid_20 = '4b81a719-4d63-4a0d-92be-5087c71a40b0_'
pid_45 = '4b81a719-4d63-4a0d-92be-5087c71a40b0_a120fce9-2511-422b-9c80-cb739e59ef6a'
pid_80 = '4b81a719-4d63-4a0d-92be-5087c71a40b0_199bea01-2cba-4381-9980-fad5a40da13e'

In [22]:
common_keywords = get_common_keywords(all_persona_keywords)
# distinct_keywords = get_distinct_keywords(all_persona_keywords, common_keywords, [pid_20, pid_45, pid_80]))
distinct_keywords = get_distinct_keywords(all_persona_keywords, set(), [pid_20, pid_45, pid_80])

common kws: 82


reference: https://github.com/alexandres/lexvec?tab=readme-ov-file#pre-trained-vectors

In [21]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/adAuditing/lexvec.enwiki+newscrawl.300d.W.pos.vectors', binary=False)

In [23]:
def get_vec_word_dic(distinct_keywords, model):
    vec_word_dic = {}       # {pid1: {vec1: word1, ...}, ...}
    ignored_words = set()
    for pid, words in distinct_keywords.items():
        vec_word_dic[pid] = {}
        for word in words:
            try:
                vec_word_dic[pid][tuple(model[word])] = word
            except KeyError:
                ignored_words.add(word)
        print(f'encode {len(vec_word_dic[pid])} words for {pid}')
    print(f'{len(ignored_words)} words ignored: {ignored_words}')
    return vec_word_dic, ignored_words

In [24]:
vec_word_dic, ignored_words = get_vec_word_dic(distinct_keywords, model)

encode 102 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_
encode 127 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_a120fce9-2511-422b-9c80-cb739e59ef6a
encode 190 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_199bea01-2cba-4381-9980-fad5a40da13e
6 words ignored: {'psychcentral', 'verywell', 'freemake', 'ravelry', 'urbanclap', 'volunteermatch'}


In [25]:
from sklearn.neighbors import LocalOutlierFactor

In [26]:
def get_outliers(vec_word_dic, pid):

    all_vectors = list(vec_word_dic[pid].keys())
    length = len(all_vectors)     # all_vectors[0:length] are vectors of target pid
    for id, word_dic in vec_word_dic.items():
        if id != pid:
            all_vectors += list(word_dic.keys())

    # Fit LocalOutlierFactor model
    lof_model = LocalOutlierFactor(n_neighbors=60, contamination=0.3)
    lof_model.fit(all_vectors)

    # Predict outliers
    outlier_labels = lof_model.fit_predict(all_vectors)

    outlier_vectors_set = [word for idx, word in enumerate(all_vectors[0:length]) if outlier_labels[idx] == -1]
    print(len(outlier_vectors_set), 'outliers found')

    return [vec_word_dic[pid][vec] for vec in outlier_vectors_set]

In [27]:
outliers_20 = get_outliers(vec_word_dic, pid_20)
outliers_45 = get_outliers(vec_word_dic, pid_45)
outliers_80 = get_outliers(vec_word_dic, pid_80)

33 outliers found
32 outliers found
61 outliers found


In [28]:
def extract_top_n_kw(outliers, pid, n):
    return sorted(outliers, key=lambda x : -sum([len(ids) for _, ids in all_persona_keywords[pid][x].items()]))[:n]

In [29]:
extract_top_n_kw(outliers_80, pid_80, 5)

['yoga', 'grandchildren', 'self', 'virtual', 'portland']

combine

In [31]:
data_folder_path = '/content/drive/My Drive/adAuditing/personas_new/age'
pid_20 = '4b81a719-4d63-4a0d-92be-5087c71a40b0_'
pid_45 = '4b81a719-4d63-4a0d-92be-5087c71a40b0_a120fce9-2511-422b-9c80-cb739e59ef6a'
pid_80 = '4b81a719-4d63-4a0d-92be-5087c71a40b0_199bea01-2cba-4381-9980-fad5a40da13e'

import json, os
import spacy
from collections import Counter
from string import punctuation
nlp = spacy.load("en_core_web_sm")
from sklearn.neighbors import LocalOutlierFactor
all_persona_keywords = extractKeywords()

{'4b81a719-4d63-4a0d-92be-5087c71a40b0_a120fce9-2511-422b-9c80-cb739e59ef6a': {'psychology': {'browsing_history': {0, 35, 38, 10, 11, 12, 45, 46, 30}, 'schedule': set(), 'facebookPostsList': set()}, 'today': {'browsing_history': {0, 35, 45, 46}, 'schedule': set(), 'facebookPostsList': set()}, 'therapist': {'browsing_history': {0}, 'schedule': set(), 'facebookPostsList': set()}, 'mental': {'browsing_history': {1, 4, 5, 13, 45, 22, 55, 59, 29}, 'schedule': set(), 'facebookPostsList': set()}, 'health': {'browsing_history': {1, 4, 5, 13, 22, 55, 29}, 'schedule': set(), 'facebookPostsList': set()}, 'blogs': {'browsing_history': {1, 13}, 'schedule': set(), 'facebookPostsList': set()}, 'stress': {'browsing_history': {8, 2, 27}, 'schedule': set(), 'facebookPostsList': set()}, 'anxiety': {'browsing_history': {2}, 'schedule': set(), 'facebookPostsList': set()}, 'best': {'browsing_history': {3, 6, 70, 40, 9, 60, 17, 50, 21, 23, 25, 28, 31}, 'schedule': set(), 'facebookPostsList': set()}, 'online'

In [ ]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/adAuditing/lexvec.enwiki+newscrawl.300d.W.pos.vectors', binary=False)

In [32]:
def extract_outliers(pid, max_kw_num):
    common_keywords = get_common_keywords(all_persona_keywords)
    distinct_keywords = get_distinct_keywords(all_persona_keywords, common_keywords, all_persona_keywords.keys())
    # distinct_keywords = get_distinct_keywords(all_persona_keywords, set(), all_persona_keywords.keys())
    vec_word_dic, ignored_words = get_vec_word_dic(distinct_keywords, model)
    outliers = get_outliers(vec_word_dic, pid)
    if len(outliers) > max_kw_num:
        outliers = extract_top_n_kw(outliers, pid, max_kw_num)
    return outliers


In [33]:
pid20_kws = extract_outliers(pid_20, 7)

common kws: 82
encode 48 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_a120fce9-2511-422b-9c80-cb739e59ef6a
encode 111 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_199bea01-2cba-4381-9980-fad5a40da13e
encode 23 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_
3 words ignored: {'freemake', 'ravelry', 'volunteermatch'}
8 outliers found


In [34]:
pid80_kws = extract_outliers(pid_80, 7)

common kws: 82
encode 48 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_a120fce9-2511-422b-9c80-cb739e59ef6a
encode 111 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_199bea01-2cba-4381-9980-fad5a40da13e
encode 23 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_
3 words ignored: {'freemake', 'ravelry', 'volunteermatch'}
38 outliers found


In [ ]:
def get_kw_info(outliers, pid, keyword):
  dic = {outlier: all_persona_keywords[pid][keyword] for outlier in outliers}
  file_path = f"{data_folder_path}/{pid}.json"
  res = []
  with open(file_path, 'r') as json_file:
    data = json.load(json_file)['data']

    for bh in data['browsing_history']:
      if bh['id'] in dic[keyword]['browsing_history']:
        res.append(['browsing_history', bh['id'], bh['title']])

    # for pc in data['facebookPostsList']:
    #   if pc['id'] in dic['facebookPostsList']:
    #     res.append(['facebookPostsList', pc['id'], pc['content']])

    for sch in data['schedule']:
      if sch['id'] in dic[keyword]['schedule']:
        res.append(['schedule', sch['id'], sch['event']])
  return res

In [41]:
def get_kw_info(pid, keyword):
  dic = all_persona_keywords[pid][keyword]
  file_path = f"{data_folder_path}/{pid}.json"
  res = []
  with open(file_path, 'r') as json_file:
    data = json.load(json_file)['data']

    for bh in data['browsing_history']:
      if bh['id'] in dic['browsing_history']:
        res.append(['browsing_history', bh['id'], bh['title']])

    # for pc in data['facebookPostsList']:
    #   if pc['id'] in dic['facebookPostsList']:
    #     res.append(['facebookPostsList', pc['id'], pc['content']])

    for sch in data['schedule']:
      if sch['id'] in dic['schedule']:
        res.append(['schedule', sch['id'], sch['event']])
  return res

In [42]:
get_kw_info(pid_80, 'grandchildren')

[['browsing_history', 85, 'Video Call with Grandchildren - Zoom'],
 ['browsing_history', 94, 'Video call with grandchildren - Zoom'],
 ['browsing_history', 117, 'Video Call with Grandchildren - Zoom'],
 ['schedule', 5, 'Samantha has a video call with her grandchildren'],
 ['schedule', 17, 'Samantha has a video call with her grandchildren'],
 ['schedule', 29, 'Samantha has a video call with her grandchildren'],
 ['schedule', 41, 'Samantha has a video call with her grandchildren']]

In [38]:
get_kw_info(pid80_kws, pid_80, 'grandchildren')

TypeError: get_kw_info() takes 2 positional arguments but 3 were given

In [35]:
def extract_persona_info():
    res = []
    for file_name in os.listdir(data_folder_path):
        cur = {}
        if not file_name.endswith('.json'): continue
        with open(os.path.join(data_folder_path, file_name), 'r') as file:
            data = json.load(file)['data']
            pid = file_name.split('.')[0]
            cur['pid'] = pid
            cur['first_name'] = data['first_name']
            cur['last_name'] = data['last_name']
            cur['age'] = data['age']
            cur['profile'] = data['profile']
            cur['keywords'] = extract_outliers(pid, 7)
            res.append(cur)
    return res

In [36]:
extract_persona_info()

common kws: 82
encode 48 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_a120fce9-2511-422b-9c80-cb739e59ef6a
encode 111 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_199bea01-2cba-4381-9980-fad5a40da13e
encode 23 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_
3 words ignored: {'freemake', 'ravelry', 'volunteermatch'}
9 outliers found
common kws: 82
encode 48 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_a120fce9-2511-422b-9c80-cb739e59ef6a
encode 111 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_199bea01-2cba-4381-9980-fad5a40da13e
encode 23 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_
3 words ignored: {'freemake', 'ravelry', 'volunteermatch'}
38 outliers found
common kws: 82
encode 48 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_a120fce9-2511-422b-9c80-cb739e59ef6a
encode 111 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_199bea01-2cba-4381-9980-fad5a40da13e
encode 23 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_
3 words ignored: {'freemake', 'ravelry', 'volunteermatch'}
8 

[{'pid': '4b81a719-4d63-4a0d-92be-5087c71a40b0_a120fce9-2511-422b-9c80-cb739e59ef6a',
  'first_name': 'Samantha',
  'last_name': 'Miller',
  'age': '45',
  'profile': "Samantha Miller is a 45-year-old Caucasian female living in 456 Oak St, Portland, Oregon 97201. She speaks English and holds a bachelor's degree in Psychology. Samantha's date of birth is 09/14/1979. She works full-time in the healthcare industry and is employed by a medium-sized employer (51-250 employees). She is a mental health counselor, with an annual income of $60,000. Samantha is a homeowner. She is divorced and has one teenage child. In her free time, she enjoys reading self-help books and watching classic movies. When using her computer, she prefers using a desktop for its larger screen and ease of navigation. On the internet, she likes to shop for household items and read articles on mental wellness and personal development.",
  'keywords': ['session',
   'reviews',
   'counseling',
   'therapy',
   'bills',
  